<a href="https://colab.research.google.com/github/mugalan/llama-index/blob/main/LLM_Indexing_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dependencies

In [1]:
import networkx as nx
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime
from pandas import to_datetime
#import plotly.figure_factory as ff
import plotly.graph_objects as go
import json
import plotly.express as px
import requests
import copy
import urllib
import pathlib
from IPython.display import HTML, display
from IPython.core.display import HTML

In [ ]:
!pip install llama-index-core #llama-parse llama-index-readers-file

In [ ]:
!pip install llama-index-llms-openai llama-index-embeddings-openai

In [9]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, Document, StorageContext, ServiceContext, load_index_from_storage
import llama_index.core.settings as settings
import nest_asyncio
nest_asyncio.apply()
OPENAI_API_KEY="open-ai-key"
openai_llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)
embed_model = OpenAIEmbedding(model="text-embedding-ada-002", api_key=OPENAI_API_KEY)

#Orise-Webservice Client

In [5]:
def get_token_omdAPI(tokenurl,credentials):
    data = {
        "username": credentials['username'],
        "password": credentials['password']
    }
    token = requests.post(tokenurl, json=data)
    return token

def call_omdAPI(webserviceurl,token,apimethod,apidata):
    postdata={"method":apimethod,"data":apidata}
    headers = {"Authorization": f"Token {token}"}
    response = requests.post(webserviceurl, json=postdata, headers=headers)
    return response

##URL Specifications

In [6]:
domain="coco-green.orise.lk"
subdomain="beta-v2/app"
tokenurl=f'https://{domain}/{subdomain}/api/api-token-auth/'
webserviceurl=f'https://{domain}/{subdomain}/api/'
ai_api_webserviceurl=f'https://{domain}/{subdomain}/api/ai-query/'
credentials={"username":"ai-user","password":"SYSCORE-demo#1234"}

In [7]:
response=get_token_omdAPI(tokenurl,credentials)
token=response.json()['token']

# Model Data Manipulation

In [8]:
apimethod = 'get_all_models'
apidata = {}
response = call_omdAPI(webserviceurl, token, apimethod, apidata)
print(response.status_code)
model_records=json.loads(response.json()['response']['data'])['records']
df=pd.DataFrame(model_records)
df

201


,app_label,model_name,fields,description
0,admin,LogEntry,"[action_time, user, content_type, object_id, o...","Model LogEntry fields are action_time, user, c..."
1,auth,Permission,"[group, user, name, content_type, codename]","Model Permission fields are group, user, name,..."
2,auth,Group,"[user, roleassignment_group, name, permissions]","Model Group fields are user, roleassignment_gr..."
3,sessions,Session,"[session_key, session_data, expire_date]","Model Session fields are session_key, session_..."
4,contenttypes,ContentType,"[logentry, permission, app_label, model]","Model ContentType fields are logentry, permiss..."
...,...,...,...,...
111,realTimeDataApp,ProductionInfoManual,"[machine_order, damaged, number_of_workers, st...",Model ProductionInfoManual fields are machine_...
112,realTimeDataApp,DownTimeInfo,"[machine_order, downtime, comments, hour]","Model DownTimeInfo fields are machine_order, d..."
113,realTimeDataApp,DamagedInfo,"[machine_order, damaged, comments, hour]","Model DamagedInfo fields are machine_order, da..."
114,realTimeDataApp,QCSampleManual,"[machine_order, counter_sample, qc_testing_wit...","Model QCSampleManual fields are machine_order,..."


##LlamaIndex

In [10]:
import llama_index

In [11]:
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core.graph_stores.simple import SimpleGraphStore
from llama_index.core import Settings

In [12]:
def initialize_index(llm, embed_model, storage_dir):
    pathlib.Path(storage_dir).mkdir(parents=True, exist_ok=True)
    fake_docstore_doc = {
            "text": "This is a placeholder document.",
            "metadata": {"data_id": "placeholder-id"},
        }
    fake_docstore = {
        "doc_id_1": fake_docstore_doc
    }

    with open(f"{storage_dir}/docstore.json", "w") as f:
        json.dump(fake_docstore, f)

    with open(f"{storage_dir}/index_store.json", "w") as f:
        json.dump(fake_docstore, f)

    documents = [
        Document(text=json.dumps(fake_docstore_doc), metadata=fake_docstore_doc)
    ]
    vector_store = SimpleVectorStore(persist_dir=storage_dir)
    graph_store = SimpleGraphStore(persist_dir=storage_dir)
    # Create a StorageContext
    storage_context = StorageContext.from_defaults(
        persist_dir=storage_dir,
        vector_store = vector_store,
        graph_store = graph_store
        )

    # Create a Vector Index, passing the storage_context during initialization
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, llm=llm, embed_model=embed_model)
    # Persist the index to the storage context
    index.storage_context.persist()
    return index


def add_records_to_index(llm, embed_model, storage_dir, records):
    # Load existing index
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    index = load_index_from_storage(storage_context, llm=llm, embed_model=embed_model)

    # Insert each document individually
    for record in records:
        text = json.dumps(record)
        metadata = record
        new_document = Document(text=text, metadata=metadata)
        index.insert(new_document)  # Insert one document at a time

    # Persist the updated index
    index.storage_context.persist()

    return index

def add_json_to_index(llm, embed_model, storage_dir, text,metadata):
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    index = load_index_from_storage(storage_context, llm=llm, embed_model=embed_model)
    new_document = Document(text=text, metadata=metadata)
    index.insert(new_document)  # Adds new document to the index
    index.storage_context.persist()  # Persists updated index
    return index

def query_index(query, llm, embed_model, storage_dir, retriever_kwargs={"top_k": 1}):
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    index = load_index_from_storage(storage_context, llm=llm, embed_model=embed_model)
    # create a query engine for the index
    query_engine = index.as_query_engine(llm=llm, retriever_kwargs=retriever_kwargs)
    response = query_engine.query(query)
    return response

##Create a model index

In [13]:
storage_dir = "/content/storage"  # Use "/content/" as the base directory
index=initialize_index(openai_llm, embed_model, storage_dir)

In [14]:
add_records_to_index(openai_llm, embed_model, storage_dir, model_records)

In [15]:
message="RegionSlotAllocation wise for week 24 to 38 for 2025" #"A Pie chart of region wise slots allocated"

In [16]:
query_response = query_index(message, openai_llm, embed_model, storage_dir)

In [17]:
query_response.metadata

{'87e050f4-6853-41a2-b52c-be77924e1ee8': {'app_label': 'modelsApp',
  'model_name': 'MaxSlotAllocation',
  'fields': ['year', 'week', 'product_type', 'slots'],
  'description': 'Model MaxSlotAllocation fields are year, week, product_type, slots'},
 '2860506b-3ddb-49db-b75e-1651d08b92cc': {'app_label': 'modelsApp',
  'model_name': 'FactorySlotAllocation',
  'fields': ['year', 'factory', 'week', 'product_type', 'slots'],
  'description': 'Model FactorySlotAllocation fields are year, factory, week, product_type, slots'}}

In [ ]:
response.text